Superstore Analysis


# 📊 STAT207 Mini-Project #1 – Superstore Sales Analysis  

**Author:** Madelyn McNamara  

---

## 1. Introduction
For this project, I analyzed the **Superstore dataset**, a popular dataset containing thousands of retail transactions. My primary research question was:  

> **How do product categories and profitability influence sales patterns over time?**  

To answer this, I focused on exploring how **sales** relate to both **numerical variables** (profit, date features) and **categorical variables** (segment, category, sub-category, region). By combining statistical analysis and visualization, I identified the variables with the strongest impact on sales and evaluated how they interact.  

This type of analysis is highly relevant in business analytics, as it can guide **pricing strategies, category management, and profitability optimization**.  

---

## 2. Dataset Overview
- **Source:** [Kaggle – Superstore Dataset](https://www.kaggle.com/datasets/vivek468/superstore-dataset-final)  
- **Shape:** 9,994 rows × 21 columns  
- **Granularity:** Each row represents a single customer order.  

### Key Variables
- **Numerical:** Sales, Profit, Discount, Quantity  
- **Categorical:** Segment, Category, Sub-Category, Region  
- **Date-related:** Order Date, Ship Date  



In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

os.makedirs("plots", exist_ok=True)


In [32]:
data = pd.read_csv("Superstore.csv", encoding='latin-1').round(1)
data.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2013-152156,09-11-2013,12-11-2013,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,262.0,2,0.0,41.9
1,2,CA-2013-152156,09-11-2013,12-11-2013,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9,3,0.0,219.6
2,3,CA-2013-138688,13-06-2013,17-06-2013,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6,2,0.0,6.9
3,4,US-2012-108966,11-10-2012,18-10-2012,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.6,5,0.4,-383.0
4,5,US-2012-108966,11-10-2012,18-10-2012,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.4,2,0.2,2.5


In [33]:
len(data)

9994

## 3. Data Cleaning

To prepare the dataset for analysis, I:

1. Converted Order Date and Ship Date to datetime format

2. Extracted Year and Month values

3. Removed outliers using the Interquartile Range (IQR) method

4. Filtered extreme values to keep Sales ≤ 300 and Profit between -25 and 45

In [34]:
# Convert dates
data['Order Date'] = pd.to_datetime(data['Order Date'], format='%d-%m-%Y')
data['Ship Date'] = pd.to_datetime(data['Ship Date'], format='%d-%m-%Y')

# Add new features
data['Year'] = data['Order Date'].dt.year
data['Month_Num'] = data['Order Date'].dt.month
data['Month'] = data['Order Date'].dt.strftime('%b')

# Keep only relevant columns
df = data[['Order Date','Sales','Profit','Year','Month_Num','Month']]

# Remove outliers with IQR
Q1, Q3 = df['Sales'].quantile([0.25, 0.75])
IQR = Q3 - Q1
lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
df = df[(df['Sales'] >= lower) & (df['Sales'] <= upper)]

Q1, Q3 = df['Profit'].quantile([0.25, 0.75])
IQR = Q3 - Q1
lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
df = df[(df['Profit'] >= lower) & (df['Profit'] <= upper)]


df = df[(df['Sales'] <= 300) & (df['Profit'].between(-25, 45))]
df.shape


(6967, 6)

## 4. Exploratory Data Analysis (EDA)


### 4.1 Distribution of Sales and Profit

In [35]:
import seaborn as sns
import matplotlib.pyplot as plt
import os

os.makedirs("plots", exist_ok=True)

sns.boxplot(data=df, x="Sales")
plt.title("Sales Distribution (Cleaned)")
plt.savefig("plots/sales_distribution.png")
plt.close()

sns.boxplot(data=df, x="Profit")
plt.title("Profit Distribution (Cleaned)")
plt.savefig("plots/profit_distribution.png")
plt.close()


Observation:

Both Sales and Profit are right-skewed.

Outlier removal reduced extreme spikes while keeping meaningful variation.

### 4.2 Correlation Between Sales and Profit

In [36]:
sns.lmplot(x="Sales", y="Profit", data=df, ci=False)
plt.title("Relationship Between Sales and Profit")
plt.savefig("plots/sales_vs_profit.png")
plt.close()

#### Observation:

Profit has a moderate positive correlation with Sales (r ≈ 0.39).

Higher sales generally lead to higher profit, but not always — suggesting discounts or cost variation.

### 4.3 Sales by Category

In [37]:
sns.boxplot(x="Category", y="Sales", data=data)
plt.title("Sales Distribution by Product Category")
plt.savefig("plots/sales_by_category.png")
plt.close()


#### Observation:

Technology products show the highest median sales.

Office Supplies typically produce smaller order amounts.

In [38]:
monthly_sales = df.groupby(['Year','Month_Num'])['Sales'].sum().reset_index()
monthly_sales['Date'] = pd.to_datetime(dict(year=monthly_sales.Year, month=monthly_sales.Month_Num, day=1))

plt.plot(monthly_sales['Date'], monthly_sales['Sales'])
plt.title("Monthly Sales Trend")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.savefig("plots/sales_trend.png")
plt.close()


#### Observation:

Sales show seasonal peaks, especially at the end of the year (holiday effect).

### 4.5 Interaction Effects


#### Category x Month

In [39]:
sns.lmplot(x='Month_Num', y='Sales', hue='Category', data=data, ci=False)
plt.title("Sales Across Months by Category")
plt.savefig("plots/sales_by_month_category.png")
plt.close()


#### Observation:

Sales trends vary by category.

Technology sales peak in later months, while Furniture and Office Supplies remain more stable.

#### Profit x Region

In [40]:
sns.lmplot(x='Profit', y='Sales', hue='Region', data=data, ci=False)
plt.title("Profit vs Sales by Region")
plt.savefig("plots/sales_profit_region.png")
plt.close()


#### Observation:

Different regions show slightly different relationships between Profit and Sales.

Suggests regional cost/pricing differences.

## 5. Statistical Modeling

I used linear regression to test predictive strength of different variable combinations:

In [41]:
import statsmodels.formula.api as smf

model1 = smf.ols("Sales ~ Profit + Category", data=data).fit()
model1.rsquared


np.float64(0.26867455212508806)

#### Results:

Sales ~ Profit + Category → R² ≈ 0.27

Sales ~ Profit + Region → R² ≈ 0.23

Sales ~ Month_Num + Category → R² ≈ 0.05

#### Takeaway:

Profit and Category together explain sales the best.

Time and region variables alone add little predictive power.

## 5. Model Improvement / Enhanced Analysis

To strengthen the predictive power of the model, we included additional features and applied transformations:

### 5.1 Feature Engineering
- Added **Discount** and **Quantity** as numerical predictors.
- Included **Sub-Category** and **Region** as categorical variables using one-hot encoding.
- Applied a **log transformation** to Sales to reduce skewness and stabilize variance.


In [42]:
import statsmodels.formula.api as smf
data = data.rename(columns={
    "Sub-Category": "Sub_Category",
    "Category": "Category",
    "Region": "Region"
})

data["Log_Sales"] = np.log1p(data["Sales"])

model_enhanced = smf.ols(
    "Log_Sales ~ Profit + Discount + Quantity + C(Category) + C(Sub_Category) + C(Region) + Profit:C(Category)",
    data=data
).fit()

print(model_enhanced.rsquared)


0.5912761073590354


#### 5.2 Results

Base model (Profit + Category): R² ≈ 0.27

Enhanced model (Profit + Discount + Quantity + Category + Sub-Category + Region + interactions): R² ≈ 0.59 ✅

##### Interpretation:

The enhanced model explains nearly 60% of the variance in Sales, showing a much stronger relationship between the predictors and sales performance.

Including additional features, interactions, and log-transformations improved the model’s predictive power substantially.

#### 5.3 Insights

Profit, Quantity, and Discounts strongly influence sales.

Sub-Category granularity shows specific products drive most revenue.

Interactions between Profit and Category reveal that high-profit items in certain categories disproportionately contribute to sales.

---

## 6. Conclusion & Future Work

### 6.1 Conclusion
The analysis demonstrates that **Profit, Category, Discount, Quantity, Sub-Category, and Region** all significantly impact sales. By enhancing the model with additional features, interaction terms, and log-transformed Sales, the R² increased from ~0.27 to **~0.59**, meaning the model now explains nearly 60% of the variation in sales.  

Key insights include:  
- High-profit items in specific categories disproportionately contribute to overall sales.  
- Discounts and quantities sold strongly influence revenue.  
- Sub-Category granularity and regional differences reveal which product types perform best in different markets.  

Overall, this model captures the complex relationships between key variables and provides a more accurate framework for understanding and predicting sales performance.

### 6.2 Future Work
There are several ways to further improve predictive accuracy and business insights:  

1. **Include Temporal Features:**  
   - Add quarterly or seasonal indicators, holidays, or rolling averages to capture trends over time.

2. **Advanced Modeling Techniques:**  
   - Explore **Random Forests, Gradient Boosting, or XGBoost** to capture nonlinear relationships.
   - Try **regularized regression (Lasso/Ridge)** to handle correlated features and prevent overfitting.

3. **Customer and Order-Level Features:**  
   - Include customer demographics or order size to refine predictions.
   - Analyze repeat customer behavior or loyalty effects on sales.

4. **Explore Profit Optimization:**  
   - Predict not only sales but also the **profit-to-sales ratio** for more actionable insights.

By incorporating these improvements, future analyses could provide even deeper insights into sales drivers and help businesses make more informed, data-driven decisions.
